# Динамическая выпуклая оболочка

Для начала, определим двумерные точки и операции их лексикографического сравнения.
Считаем, что $p < q$, если $p_y < q_y \vee p_y = q_y \wedge p_x < q_x$

In [18]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __lt__(self, other):
        return self.y < other.y or self.y == other.y and self.x <  other.x
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    def __le__(self, other):
        return self < other or self == other
    def __gt__(self, other):
        return other < self
    def __ge__(self, other):
        return other < self or self == other
    def __ne__(self, other):
        return not self == other

Мы хотим получить алгоритм, который мог бы поддерживать выпуклую оболочку в онлайне с возможностью добавления, удаления и локализации точки в выпуклой оболочке.
Здесь приведен алгоритм делающий эти три операции за $O(log^2n)$.

Для начала, определим левую и правую выпуклую оболочку множества точек $P$.


## Определение

Левой (Правой) выпуклой оболочкой множества точек $P$ называется выпуклая оболочка множества точек $P\cup \left\{(+\infty, 0)\right\}$ ($P\cup \left\{(-\infty, 0)\right\}$)

Здесь и далее Будем реализовывать левую выпуклую оболочку, в правой все так же с точностью до знака

Для начала, определим структуры данных, в которых у нас все будет храниться (методы добавим в них позже).
Мы будем хранить отсортированные лексикографически точки в листьях сбалансированного бинарного дерева поиска.
Во внутренних вершинах будем хранить вспомогательную информацию: 
во-первых, наименьшую точку в поддереве с корнем в данной вершине; а во-вторых, 
мост между выпуклыми оболочками точек левого и правого поддеревьев данной вершины.

<table style="border-style: hidden; border-collapse: collapse;">
<tr style="border-style: hidden; border-collapse: collapse;">
    <td style="border-style: hidden; border-collapse: collapse;">
        <div><img src="img/bridge.png"><center>Рис 1. мост между выпуклыми оболочками</center></div>
    </td>
    <td style="border-style: hidden; border-collapse: collapse;">
        <div><img src="img/left-right.png"><center>Рис 2. Левая и правая выпуклые оболочки</center></div>
    </td>
</tr>
</table>

Brdge - определяет мост между двумя выпуклыми оболочками:

In [12]:
class Bridge:
    def __init__(self, p, q):
        self.left = p
        self.right = q

Node - вершина нашего дерева:

In [17]:
class Node:
    def __init__(self, point, parent=None, left=None, right=None, bridge=None):
        self.point = point
        self.parent = parent
        self.left = left
        self.right = right
        self.bridge = bridge
    
    def leaf(self):
        return self.left is not None or self.right is not None

## Объединение выпуклых оболочек

Пусть, мы хотим объединить оболочки $A$ и $C$, но, для начала наложим на них ограничение - должна существовать горизонтальная прямая, которая их разделяет (как на рисунке 1). Мы можем воспользоватся идеями бинарного поиска для нахождения моста. Для этого необходим критерий спуска, по которому мы будем определять подотрезок, до которого нужно сузить задачу, имея точки $p \in A$ и  $q \in C$, определяющие общую касательную к выпуклым оболочкам множеств $A$ и  $C$.

Обозначим $CH(Q)$ - выпуклая оболочка множества $Q$,

$p^+ = \min\limits_{p' \in \left\{CH(A) > p\right\}} p'$, $p^+$ - верхняя грань $p$ из множества $CH(A)$,<br>
$p^- = \max\limits_{p' \in \left\{CH(A) < p\right\}} p'$, $p^-$ - нижняя грань $p$ из множества $CH(A)$,<br>
аналогично определяются $q^+$ и $q^-$.
Так же, определим поворот вектора $\overline{ab}$ с точкой $c$, как функцию $rot(\overline{ab}, c)$.
Теперь, определим отношение между вектором и точкой "справа" - $rot(\overline{ab}, c) > 0$ (точка справа от вектора).

Далее, рассмотрим все возможные случаи расположения этих точек.

1. Точки $p^+, p^-, q^+, q^-$ лежат справа от вектора $\overline{qp}$ - следовательно, отрезок $qp$ - искомый мост.
2. $p^+, p^-, q^+$ - справа, а $q^+$ - слева от $\overline{qp}$. Тогда отбрасываем $CH(A) < p$ и $CH(C) > q$.
3. $p^{+}$, $p^{-}$, $q^{-}$ — справа, $q^{+}$ — слева от $\overline{qp}$. Отбрасываем $CH(A) < p$ и $CH(C) < q$.
4. Как в случае 2, только слева от $\overline{qp}$ находится точка $p^{+}$. Отбрасываем $CH(A) < p$ и $CH(C) > q$.
5. Как в случае 3, только слева от $\overline{qp}$ находится точка $p^{-}$. Отбрасываем $CH(A) > p$ и $CH(C) > q$.
6. $p^{-}$, $q^{+}$ — справа, $p^{+}$, $q^{-}$ — слева от $\overline{qp}$. Отбрасываем $CH(A) < p$ и $CH(C) > q$.
7. $p^{+}$, $q^{+}$ — справа, $p^{-}$, $q^{-}$ — слева от $\overline{qp}$. Отбрасываем $CH(C) > q$.
8. $p^{-}$, $q^{-}$ — справа, $p^{+}$, $q^{+}$ — слева от $\overline{qp}$. Отбрасываем $CH(A) < p$.
9. $p^{+}$, $q^{-}$ — справа, $p^{-}$, $q^{+}$ — слева от $\overline{qp}$. Этот случай дробится на два подслучая.

Вот как можно схематично показать все случаи:
<div>
<table>
    <tr>
        <td><div><img src="img/case1.png" width=300 height=300><center>Рис 3. Случай 1</center></div></td>
        <td><div><img src="img/case2.png" width=300 height=300><center>Рис 4. Случай 2</center></div></td>
        <td><div><img src="img/case3.png" width=300 height=300><center>Рис 5. Случай 3</center></div></td>
    </tr>
    <tr>
        <td><div><img src="img/case4.png" width=300 height=300><center>Рис 6. Случай 4</center></div></td>
        <td><div><img src="img/case5.png" width=300 height=300><center>Рис 7. Случай 5</center></div></td>
        <td><div><img src="img/case6.png" width=300 height=300><center>Рис 8. Случай 6</center></div></td>
    </tr>
    <tr>
        <td><div><img src="img/case7.png" width=300 height=300><center>Рис 9. Случай 7</center></div></td>
        <td><div><img src="img/case8.png" width=300 height=300><center>Рис 10. Случай 8</center></div></td>
        <td><div><img src="img/case9.png" width=300 height=300><center>Рис 11. Случай 9</center></div></td>
    </tr>
</table>
</div>

Рассмотрим подслучаи случая 9. Пусть $m$ — прямая, разбивающая $P$ на $A$ и $C$. Пусть также $l_p$ и $l_q$ — касательные к выпуклым оболочкам в точках $p$ и $q$ соответственно. Если $s$ — точка пересечения $l_p$ и $l_q$ лежит ниже прямой $m$, то точка пересечения моста и выпуклой оболочки $A$ может лежать в треугольнике, образованном прямыми $l_p$, $l$ и $m$, или выше $p$. Тогда можем удалить часть $C$ ниже $q$ (см. рис. 12.1). Случай, когда $s$ лежит выше $m$, аналогичен (см. рис. 12.2).

<table style="border-style: hidden; border-collapse: collapse;">
<tr style="border-style: hidden; border-collapse: collapse;">
    <td style="border-style: hidden; border-collapse: collapse;">
        <div><img src="img/case9.1.png"><center>Рис 12.1. мост между выпуклыми оболочками</center></div>
    </td>
    <td style="border-style: hidden; border-collapse: collapse;">
        <div><img src="img/case9.2.png"><center>Рис 12.2. Левая и правая выпуклые оболочки</center></div>
    </td>
</tr>
</table>

## Получение множества точек

## Локализация

## Вставка

## Удаление

In [19]:
import numpy as np